In [1]:
#%tensorflow_version 1.x

Importing Libraries

In [2]:
import tensorflow as tf

from keras import Sequential
from keras.layers import Dense, Reshape, Dropout

Importing pre-trained model

In [3]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])
# print(embeddings)

Fine-tuning with custom layers

In [4]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", output_shape=[128],
                           input_shape=[], dtype=tf.string)
model = Sequential()
model.add(hub_layer)

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


LSTM Layer (Fine-tuning) 

In [5]:
# from keras.layers import LSTM, Bidirectional,
# hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1",
#                            output_shape=[128],
#                            input_shape=[],
#                            dtype=tf.string)
# model = Sequential()
# model.add(hub_layer)
# model.add(Reshape((128,1)))
# model.add(Bidirectional(LSTM(128, activation='relu')))
# # model.add(LSTM(64, activation='relu'))
# # model.add(Dense(32, activation='relu'))
# # model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.summary()

CNN (Fine-tuning)

In [6]:
# from keras.layers import Conv1D, MaxPooling1D, LSTM
# hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1",
#                            output_shape=[128],
#                            input_shape=[],
#                            dtype=tf.string)

# model = Sequential()
# model.add(hub_layer)
# model.add(Reshape((128,1)))
# model.add(Dropout(0.2))
# model.add(Conv1D(64, 5, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))

In [7]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path = "drive/My Drive/reddit_data.csv"
df = pd.read_csv(path, encoding='ISO-8859-1')

df.columns = ['title', 'data']
df["post"] = df["title"] + df["data"]
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,title,data,post
0,New wiki on how to avoid accidentally encourag...,We've been seeing a worrying increase in pro-s...,New wiki on how to avoid accidentally encourag...
1,Please remember that NO ACTIVISM of any kind i...,"Activism, i.e. advocating or fundraising for s...",Please remember that NO ACTIVISM of any kind i...
2,SuiCiDe iS a PerManeNt SolUtiOn tO a TemPorAry...,oh my god fuck off. i know they mean well but ...,SuiCiDe iS a PerManeNt SolUtiOn tO a TemPorAry...
3,Who the fuck created us and why do they want t...,I keep seeing new posts in this thread daily. ...,Who the fuck created us and why do they want t...
4,Am I the only person on this planet without an...,"I didnât even ask to be this way, I wish I w...",Am I the only person on this planet without an...


In [8]:
positive_posts = df.iloc[:989,2]
positive_posts = list(positive_posts)

negative_posts = df.iloc[989:,2]
negative_posts = list(negative_posts)

x = len(positive_posts)
y = len(negative_posts)
print(x, y)

train_pos = positive_posts[(x//5):]
test_pos = positive_posts[:(x//5)]
train_neg = negative_posts[(y//5):]
test_neg = negative_posts[:(y//5)]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg
train_x = np.array(train_x)
test_x = np.array(test_x)
print("train_x.shape = " + str(len(train_x)))
print("test_x.shape = " + str(len(test_x)))
train_y = np.append(np.zeros((len(train_pos), 1)), np.ones((len(train_neg), 1)), axis=0)
test_y = np.append(np.zeros((len(test_pos), 1)), np.ones((len(test_neg), 1)), axis=0)

989 2000
train_x.shape = 2392
test_x.shape = 597


Model Training 

In [9]:
model.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [10]:
# train_x = tf.convert_to_tensor(train_x)
# train_y = tf.convert_to_tensor(train_y)
# test_x = tf.convert_to_tensor(test_x)
# test_y = tf.convert_to_tensor(test_y)
model.fit(train_x, train_y, epochs=5, batch_size=32, verbose=2)
results = model.evaluate(test_x, test_y)
for name,value in zip(model.metrics_names, results):
  print(f'{name}: {value}')

Epoch 1/5
75/75 - 1s - loss: 0.3871 - accuracy: 0.9235
Epoch 2/5
75/75 - 0s - loss: 0.2183 - accuracy: 0.9553
Epoch 3/5
75/75 - 0s - loss: 0.1920 - accuracy: 0.9544
Epoch 4/5
75/75 - 0s - loss: 0.1759 - accuracy: 0.9582
Epoch 5/5
75/75 - 0s - loss: 0.1654 - accuracy: 0.9615
19/19 [==============================] - 0s 2ms/step - loss: 0.1420 - accuracy: 0.9514
loss: 0.14203818142414093
accuracy: 0.9514237642288208


Model Testing

In [11]:
model.predict(np.array(["is it strange when a part of you wishes to get corona with the hope to die?",
                        "Why do I even fucking bother anymore. My existence is a mistake.",
                        "depression is so frustrating.",
                        "Tricking the novel coronavirus with a fake “handshake”: Scientists have found a way to trick the novel coronavirus into binding with a protein fragment that resembles a friendly receptor, effectively inactivating SARS-CoV-2 before it can infect a cell.",
                        "I know everyone despised CoD ghosts but let’s be honest this was the funniest and best thing ever added to a CoD game ever!",
                        "Orange deck of playing cards cut into a 3D design using just an xacto knife. This is the most detailed cut of a deck of playing cards I have done so far.",
                        "This ice-cream is good",
                        "I am Happy today",
                        "you guys have no idea how beautiful it has been to have someone come out of blue and help me fix my broken heart..last week I was in a deep hole in my depression wanting to kill myself..and fast forward now I’m extremely happy..it’s true when they say to trust the process",
                        "If I don’t talk to y’all the next few days, it’s because not being in school is going to give me way too much time to think and I’m most definitely going to overthink my entire life and enter a very depressive state. Thanks for understanding.",
                        "I don't want to live anymore",
                        "So i started working two and a half years ago, everybody was always asking me if im fine, and i always say, yeah sure. But to be honest, im sick of all this shit, this society is all about money, nobody wanna really hear what you got to say, my parents are always judging me, my whole life they where almost never emotional on my side, so i got no relation to them, even tho i got to act like that, cause society doesnt want anything else, im sick off all this weight on my shoulders, this pressure, im was a happy person a long time ago, now i am most of the time down and depressed on the inside, but happy on the outside. One of the only things that really does some kind fun is smoking weed, because my mind is so focused on the moment, and i go out into nature and dont think about other humans anymore. Im only 18, but i feel like im sick of my life already, most of the time i just want to end my current life",
                        "I hate everything about this world. I want to end it so badly."]))

array([[0.24444032],
       [0.09525368],
       [0.21883246],
       [0.37422788],
       [0.25611493],
       [0.557194  ],
       [0.35874435],
       [0.0972029 ],
       [0.23928684],
       [0.29990655],
       [0.0693236 ],
       [0.02023298],
       [0.08258957]], dtype=float32)

In [18]:
model.predict(np.array(["Can't wait to have fun tomorrow. It's going to be best day of my life",
                        "Can't live anymore",
                        "I am Happy today",
                        "Messi is the best player on this planet and there is no doubt about it",
                        "India is the best country in the world"]))

array([[0.11687911],
       [0.07356706],
       [0.10656896],
       [0.3718497 ],
       [0.42825615]], dtype=float32)

BERT model

In [13]:
# !pip install transformers
# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures

# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [14]:
# model.summary()

In [15]:
# import tensorflow as tf
# import pandas as pd